In [1]:
from tensorflow.keras import Input, layers

input_tensor = Input(shape=(32,))
dense = layers.Dense(32, activation='relu')

output_tensor = dense(input_tensor)

# 다중 입력 모델

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras import layers, Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [5]:
import numpy as np
from tensorflow.keras.utils import to_categorical

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))

question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))

answers = np.random.randint(0, answer_vocabulary_size, size=num_samples)

answers = to_categorical(answers)

model.fit([text, question], answers, epochs=10, batch_size=128)

model.fit({'text': text, 'question':  question}, answers, epochs=10, batch_size=128)

Epoch 1/10
8/8 [==============================] - 5s 118ms/step - loss: 6.2143 - acc: 0.0000e+00
Epoch 2/10
8/8 [==============================] - 1s 116ms/step - loss: 6.1914 - acc: 0.0330
Epoch 3/10
8/8 [==============================] - 1s 115ms/step - loss: 6.1116 - acc: 0.0030
Epoch 4/10
8/8 [==============================] - 1s 113ms/step - loss: 6.0366 - acc: 0.0080
Epoch 5/10
8/8 [==============================] - 1s 115ms/step - loss: 5.9752 - acc: 0.0140
Epoch 6/10
8/8 [==============================] - 1s 119ms/step - loss: 5.9078 - acc: 0.0090
Epoch 7/10
8/8 [==============================] - 1s 114ms/step - loss: 5.8389 - acc: 0.0100
Epoch 8/10
8/8 [==============================] - 1s 115ms/step - loss: 5.7555 - acc: 0.0120
Epoch 9/10
8/8 [==============================] - 1s 116ms/step - loss: 5.6575 - acc: 0.0110
Epoch 10/10
8/8 [==============================] - 1s 115ms/step - loss: 5.5804 - acc: 0.0140
Epoch 1/10
8/8 [==============================] - 3s 117ms/step -

# 다중 출력 모델

In [8]:
from tensorflow.keras import layers, Input
from tensorflow.keras.models import Model
vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups,
                                 activation='softmax',
                                 name='income')(x)
age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups,
                                 activation='softmax',
                                 name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input,
              [age_prediction, income_prediction, gender_prediction])

In [10]:
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy','binary_crossentropy'])

# 위와 동일하지만 출력 층에 이름을 지정했을 때 사용할 수 있음
model.compile(optimizer='rmsprop',
              loss={'age':'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'})

In [11]:
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy','binary_crossentropy'],
              loss_weights=[0.25, 1., 10.])

# 위와 동일하지만 출력 층에 이름을 지정했을 때 사용할 수 있음
model.compile(optimizer='rmsprop',
              loss={'age':'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'},
              loss_weights={'age': 0.25,
                            'income': 1.,
                            'gender': 10.})

In [ ]:
# model.fit(posts, [age_targets, income_targets, gender_targets],
#           epochs=10, batch_size=64)

# 위와 동일하지만 출력 층에 이름을 지정했을 때 사용할 수 있음
# model.fit(posts, {'age': age_targets,
#                   'income': income_targets,
#                   'gender': gender_targets},
#           epochs=10, batch_size=64)

# inception V3 모듈의 예

In [ ]:
from tensorflow.keras import layers
branch_a = layers.Conv2D(128, 1, activation='relu', strides=2)(x)

branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, activation='relu')(branch_b)

branch_c = layers.AveragePooling2D(3, strides=2)(x)
branch_c = layers.Conv2D(128, 3, activation = 'relu')(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', stride=2)(branch_d)
output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)

# 잔차 연결의 예

## 특성 맵의 크기가 같을 때 원본을 그대로 사용

In [ ]:
from tensorflow.keras import layers

x = ...
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)

y = layers.add([y,x])

## 특성 맵의 크기가 다를 때 선형 변환을 사용

In [ ]:
from tensorflow.keras import layers

x = ...
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)

residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)

y = layers.add([y, residual])

# 층 가중치 공유층 사용하는 모델을 구현하는 예

In [ ]:
from tensorflow.keras import layers, Input
from tensorflow.keras.models import Model

lstm = layers.LSTM(32)
left_input = Input(shape=(None, 128))
left_output = lstm(left_input)

right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)

model = Model([left_input, right_input], predictions)
model.fit([left_data, right_data], targets)

# 샴 비전 모델 구현 예

In [16]:
from tensorflow.keras import layers, Input
from tensorflow.keras.applications import Xception

xception_base = Xception(weights=None,
                         include_top=False)

left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

left_features = xception_base(left_input)
right_features = xception_base(right_input)

merged_features = layers.concatenate([left_features, right_features], axis=-1)